In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('因子報酬敘述統計/factor_return_adj.csv').drop('Unnamed: 0',axis=1)
data = data.set_index('Date')


#半衰期(half-life)權重
def half(t,a):  #t=half-life, a=sample size
     lambd=0.5**(1/t)
     weightlist=[]
     for i in range(a-1,-1,-1): #a=539,b=-1,i=539、538、...0
         halflife=lambd**i
         weightlist.append(halflife)
     weightlist = np.array(weightlist)/sum(weightlist)   
     return weightlist

delta_weilist=[]
for i in range(1,16):
    delta_wei=(1-(i/(1+15)))
    delta_weilist.append(delta_wei)
    
#sam_size=1500
sam_size=540
#half=125
half_ =45 

In [2]:
data

,Market,Computing,Culture & Entertainment,Currency,DeFi,Digitization,Smart Contract Platform,centrality,liquidity,momentum,size,volatility
Date,,,,,,,,,,,,
2017-07-17,0.091373,-0.012786,0.043789,-0.028766,0.116875,-0.059013,0.056750,0.002436,0.017679,-0.036450,0.020141,-0.017323
2017-07-18,-0.078039,-0.000949,-0.060440,0.016575,-0.198687,-0.003590,-0.021128,0.003413,-0.047084,0.029072,-0.014726,-0.003133
2017-07-19,0.173544,-0.016747,-0.034344,0.026725,-0.340820,-0.107064,-0.035929,0.022197,-0.118633,-0.002559,0.029545,0.037455
2017-07-20,-0.039258,-0.009316,0.027116,0.002775,-0.022398,0.057073,-0.009591,-0.007452,0.015713,0.001289,-0.016713,0.018122
2017-07-21,0.057236,-0.021485,0.042302,0.001407,-0.011096,0.105064,-0.009307,0.023237,0.028209,-0.028937,-0.020109,0.009425
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-22,0.034675,0.000036,-0.003695,-0.003959,-0.004391,0.025013,0.008425,0.009655,0.004005,-0.001929,-0.001126,-0.001348
2022-11-23,0.001508,0.006352,-0.041809,0.010070,0.002532,-0.028334,-0.002219,0.017206,0.017392,0.002785,0.000030,-0.017493
2022-11-24,-0.003465,-0.005081,-0.007847,0.002885,0.002120,-0.009950,-0.000668,0.006288,0.004339,0.006472,-0.000133,-0.010573


In [82]:
sum(half(half_,sam_size))

1.0

In [83]:
mat = data.shape[1]

deltalist_pp=[]
deltalist_mm=[]
for day in range(data.shape[0]-sam_size):
    data1=data.iloc[0+day:sam_size+day]
    deltalist_m=[]
    deltalist_p=[]
    for i in range(1,16): 
        Wv = np.zeros((sam_size,sam_size))
        Wv[np.arange(sam_size), np.arange(sam_size)] = half(half_,sam_size)
        Wc = np.zeros((sam_size,sam_size))
        Wc[np.arange(sam_size), np.arange(sam_size)] = half(half_,sam_size)
        Wv = Wv[i:,i:]
        Wc = Wc[i:,i:]
        M1 = data1.shift(i)[i:len(data1)] #0-538
        M2 = data1[i:len(data1)]          #1-539
        U1 = np.zeros((mat,mat))
        U1[np.arange(mat), np.arange(mat)] = 1
        U2 = np.ones((mat,mat))
        U2[np.arange(mat), np.arange(mat)] = 0
        Cd_p = Wv.dot(M1).T.dot(M2)*U1 + Wc.dot(M1).T.dot(M2)*U2
        Cd_m = Cd_p.T
        deltalist_p.append(Cd_p) 
        deltalist_m.append(Cd_m)
    deltalist_pp.append([data.index[sam_size+day],deltalist_p])
    deltalist_mm.append([data.index[sam_size+day],deltalist_m])
    print(day)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [84]:
port_varlist=[]
for day in range(data.shape[0]-sam_size):
    data1=data.iloc[0+day:sam_size+day]
    data2=data1-data1.describe().loc['mean']
     
    #EUE3S_var
    varT=(data2.T*half(half_,sam_size)).dot(data2)
    varT=varT/sum(half(half_,sam_size))
    var=np.diag(varT)
    
    #EUE3S_cor
    corT=(data2.T*half(half_,sam_size)).dot(data2)
    corT=corT/sum(half(half_,sam_size))
    #Cd
    Cd=corT.values.copy()
    np.fill_diagonal(Cd, var)
    #CdT=pd.DataFrame(Cd)
    port_varlist.append([data.index[sam_size+day],Cd])
    print(day) #0、1、...2262 #0、1、...1302

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [85]:
Cmlist=[]    
for day in range(data.shape[0]-sam_size):  
    delta_times=[]
    for i in range(15): 
        d_times=delta_weilist[i]*( deltalist_pp[day][1][i]+ deltalist_mm[day][1][i] )
        delta_times.append(d_times)
    delta_sum=sum(delta_times)   
    Cm=(port_varlist[day][1] + delta_sum)*22
    Cmlist.append([data.index[sam_size+day], Cm])

In [86]:
len(Cmlist)

1220

In [87]:
for i in range(data.shape[0]-sam_size):
    a=Cmlist[i][1]
    np.save('Common Factor Risk_Long\Cm_Strong Momentum'+str(i)+'.npy',a)